In [0]:
from google.colab import drive
drive.mount("/content/drive")

In [0]:
from __future__ import print_function, division

import csv
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from google.colab.patches import cv2_imshow
from skimage import io, transform
import torch.nn.functional as F
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import SequentialSampler,Dataset,DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import os
import copy
import cv2
import glob

In [0]:
def readCsv(header,data,name):
  with open(name, 'wt') as f:
    csvPrinter = csv.writer(f, quoting=csv.QUOTE_ALL)
    csvPrinter.writerow(header)
    csvPrinter.writerows(data)

In [0]:



Annotations = sorted([k for k in os.listdir('/content/drive/My Drive/dataset/annotations/') 
                   if os.path.isfile(os.path.join('/content/drive/My Drive/dataset/annotations/', k))])

DataAnn = []
for ann in Annotations:
  directory = '/content/drive/My Drive/dataset/annotations/' + ann
  file = open(directory, "r").read().split("\n")
  for line in range(len(file)-1):
      values = file[line].split()
      values2 = file[line+1].split()[1:]
      data = [ann[:-4]] + values + values2
      DataAnn.append(data)
      if line == len(file)-3:
        break
    
trains = os.listdir ('/content/drive/My Drive/dataset/videos/val/')

DataTrain = []
for i in trains:
  file = sorted([j for j in os.listdir('/content/drive/My Drive/dataset/videos/val/' + i) 
                 if os.path.isfile(os.path.join('/content/drive/My Drive/dataset/videos/val/' + i, j))])
  t0 = ""
  t1 = ""
  for frame in file:
    if t0!= "":
      if t1 != "":        
        t0 = t1
        t1 = frame
      else:
        t1 = frame
      data = [i] +[int(t0[:-4])] + [t0] + [t1] 
      DataTrain.append(data)
    else:
      t0 = frame

header = ['video', 'frame_id', 'x1', 'y1', 'x2', 'y2','x11','y11','x22','y22']
readCsv(header,DataAnn,"Annos.csv")
header = ['video','frame_id', 'frame1','frame2']
readCsv(header,DataTrain,"Test.csv")
annos = pd.read_csv("Annos.csv")
tests = pd.read_csv("Test.csv")
merged = pd.merge(annos,tests)
merged.head()
merged.to_csv("/content/drive/My Drive/mergedValidation.csv", index=False)

In [0]:
Annotations = sorted([k for k in os.listdir('/content/drive/My Drive/dataset/annotations/') 
                   if os.path.isfile(os.path.join('/content/drive/My Drive/dataset/annotations/', k))])
DataAnn = []
for ann in Annotations:
  directory = '/content/drive/My Drive/dataset/annotations/' + ann
  file = open(directory, "r").read().split("\n")
  for line in range(len(file)-1):
      values = file[line].split()
      values2 = file[line+1].split()[1:]
      data = [ann[:-4]] + values + values2
      DataAnn.append(data)
      if line == len(file)-3:
        break
trains = os.listdir ('/content/drive/My Drive/dataset/videos/train/')

DataTrain = []
for i in trains:
  file = sorted([j for j in os.listdir('/content/drive/My Drive/dataset/videos/train/' + i)
                 if os.path.isfile(os.path.join('/content/drive/My Drive/dataset/videos/train/' + i, j))])
  t0 = ""
  t1 = ""
  for frame in file:
    if t0!= "":
      if t1 != "":        
        t0 = t1
        t1 = frame
      else:
        t1 = frame
      data = [i] +[int(t0[:-4])] + [t0] + [t1] 
      DataTrain.append(data)
    else:
      t0 = frame
header = ['video', 'frame_id', 'x1', 'y1', 'x2', 'y2','x11','y11','x22','y22']
readCsv(header,DataAnn,"Annos.csv")
header = ['video','frame_id', 'frame1','frame2']
readCsv(header,DataTrain,"Test.csv")
annos = pd.read_csv("Annos.csv")
tests = pd.read_csv("Test.csv")
merged = pd.merge(annos,tests)
merged.head()
merged.to_csv("/content/drive/My Drive/mergedTrain.csv", index=False)

In [0]:
Annotations = sorted([f for f in os.listdir('/content/drive/My Drive/dataset/annotations/') 
                   if os.path.isfile(os.path.join('/content/drive/My Drive/dataset/annotations/', f))])

DataAnn = []
for ann in Annotations:
  file = open('/content/drive/My Drive/dataset/annotations/' + ann, "r").read().split("\n")
  for line in range(len(file)-1):
      values = file[line].split()
      data = [ann[:-4]] + values
      DataAnn.append(data)
    
trains = os.listdir ('/content/drive/My Drive/dataset/videos/test/')

DataTrain = []
for i in trains:
  file = sorted([k for k in os.listdir('/content/drive/My Drive/dataset/videos/test/' + 
                                      i) if os.path.isfile(os.path.join('/content/drive/My Drive/dataset/videos/test/' + i, k))])
  t0 = ""
  t1 = ""
  for frame in file:
    if t0!= "":
      if t1 != "":        
        t0 = t1
        t1 = frame
      else:
        t1 = frame
      data = [i] +[int(t0[:-4])] + [t0] + [t1] 
      DataTrain.append(data)
    else:
      t0 = frame

header = ['video', 'frameId', 'x1', 'y1', 'x2', 'y2']
readCsv(header,DataAnn,"Annos.csv")
header = ['video','frame_id', 'frame1','frame2']
readCsv(header,DataTrain,"Test.csv")

annos = pd.read_csv("Annos3.csv")
tests = pd.read_csv("Test.csv")
merged = pd.merge(annos,tests)
merged.head()
merged.to_csv("/content/drive/My Drive/mergedTest.csv", index=False)

In [0]:
def normalizeVector(list):
    vector = []
    for x in list:
        normalized = (x-min(list))/(max(list)-min(list))
        vector.append(normalized)
    return vector

In [0]:
class TrainDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir 
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        video_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        frame_name = os.path.join(video_name,
                                self.landmarks_frame.iloc[idx, 10])
        image = io.imread(frame_name)
        frame2_name = os.path.join(video_name,
                                self.landmarks_frame.iloc[idx, 11])
        image2 = io.imread(frame2_name)
        landmarks = self.landmarks_frame.iloc[idx, 2:6].values
        
        landmarks2 = self.landmarks_frame.iloc[idx, 6:10].values
        
        img = np.array(image)
        img2 = np.array(image2)
        
        
        coordinates = []
        coordinates2 = []
        x1 = landmarks[0]
        y1 = landmarks[1]
        x2 = landmarks[2]
        y2 = landmarks[3]
        
        x11 = landmarks2[0]
        y11 = landmarks2[1]
        x22 = landmarks2[2]
        y22 = landmarks2[3]
        box_h = y2 - y1
        box_w = x2 - x1
        bbox = patches.Rectangle((x11, y11), x22-x11, y22-y11,
             linewidth=2, facecolor='none',edgecolor = "white")
        
        if int(y1-(box_h/2))<0 and int(x1-(box_w/2))<0:
          croppedImage1 = img[0:int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[0:int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedX1 = 0
          croppedY1 = 0
        elif int(y1-(box_h/2))<0:
          croppedImage1 = img[0:int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[0:int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedY1 = 0
          croppedX1 = x1-box_w/2
        elif int(x1-(box_w/2))<0:
          croppedImage1 = img[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedX1 = 0
          croppedY1 = y1-box_h/2
        elif int(y1-(box_h/2))>=0 and int(x1-(box_w/2))>=0:
          croppedImage1 = img[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedX1 = x1-box_w/2
          croppedY1 = y1-box_h/2
        
  
        croppedX2 = x1+box_w/2
        croppedY2 = y1+box_h/2
      
        nx1 = x1 - croppedX1
        nx2 = x2 - croppedX1
        ny1 = y1 - croppedY1
        ny2 = y2 - croppedY1
      
        nx1 = nx1 * (224/len(croppedImage1[0]))
        nx2 = nx2 * (224/len(croppedImage1[0]))
        ny1 = ny1 * (224/len(croppedImage1))
        ny2 = ny2 * (224/len(croppedImage1))
      
        box_w1 = nx2-nx1
        box_h1 = ny2-ny1
      
        nx11 = x11 - croppedX1
        nx22 = x22 - croppedX1
        ny11 = y11 - croppedY1
        ny22 = y22 - croppedY1
        
        nx11 = nx11 * (224/len(croppedImage2[0]))
        nx22 = nx22 * (224/len(croppedImage2[0]))
        ny11 = ny11 * (224/len(croppedImage2))
        ny22 = ny22 * (224/len(croppedImage2))
        
        
        
        croppedX2 = croppedX2 - croppedX1
        croppedY2 = croppedY2 - croppedY1
        croppedX1 = 0
        croppedY1 = 0
  
        box_w2 = nx22-nx11
        box_h2 = ny22-ny11
      
      
        
        coordinates.append(nx1)
        coordinates.append(ny1)
        coordinates.append(nx2)
        coordinates.append(ny2)
        
        coordinates2.append(nx11)
        coordinates2.append(ny11)
        coordinates2.append(nx22)
        coordinates2.append(ny22)
        bbox2 = patches.Rectangle((nx11, ny11), box_w2, box_h2,
             linewidth=2, facecolor='none',edgecolor = "white")
        
        croppedImage1 = self.transform(croppedImage1)
        croppedImage2 = self.transform(croppedImage2)
        x = torchvision.utils.make_grid(croppedImage1, nrow=5).permute(1, 2, 0)
        y = torchvision.utils.make_grid(croppedImage2, nrow=5).permute(1, 2, 0)
#         plt.figure()
#         plt.title(video_name)
#         plt.imshow(x)
#         plt.figure()
#         plt.title(video_name)
#         plt.imshow(y)
#         plt.figure()
#         fig, ax = plt.subplots(1)
#         ax.add_patch(bbox)
#         plt.imshow(img2)
#         plt.text(nx11, y11, s=str(nx11)+","+str(ny11), 
#                 color='white', verticalalignment='top',
#                 bbox={'color': "green", 'pad': 0})
#         plt.axis('off')
#         plt.show()
  
        
        coordinates = torch.from_numpy(np.array(coordinates))
        coordinates2 = torch.from_numpy(np.array(coordinates2))
      
        sample1 = (x,y,coordinates,coordinates2)
        return sample1

In [0]:
def cropImage(image,x1,x2,y1,y2):
        box_h = y2 - y1
        box_w = x2 - x1
        if int(y1-(box_h/2))<0 and int(x1-(box_w/2))<0:
          croppedImage = image[0:int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedX1 = 0
          croppedY1 = 0
        elif int(y1-(box_h/2))<0:
          croppedImage = image[0:int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedY1 = 0
          croppedX1 = x1-box_w/2
        elif int(x1-(box_w/2))<0:
          croppedImage = image[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedX1 = 0
          croppedY1 = y1-box_h/2
        elif int(y1-(box_h/2))>=0 and int(x1-(box_w/2))>=0:
          croppedImage = image[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedX1 = x1-box_w/2
          croppedY1 = y1-box_h/2
          
        croppedX2 = x1+box_w/2
        croppedY2 = y1+box_h/2
      
        nx1 = x1 - croppedX1
        nx2 = x2 - croppedX1
        ny1 = y1 - croppedY1
        ny2 = y2 - croppedY1
        return croppedImage,nx1,nx2,ny1,ny2,croppedX1,croppedX2,croppedY1,croppedY2

In [0]:
class ValidDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir 
        self.transform = transform
        

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        video_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        frame_name = os.path.join(video_name,
                                self.landmarks_frame.iloc[idx, 10])
        image = io.imread(frame_name)
        frame2_name = os.path.join(video_name,
                                self.landmarks_frame.iloc[idx, 11])
        image2 = io.imread(frame2_name)
        landmarks = self.landmarks_frame.iloc[idx, 2:6].values
        
        landmarks2 = self.landmarks_frame.iloc[idx, 6:10].values
        
        img = np.array(image)
        img2 = np.array(image2)
        coordinates = []
        coordinates2 = []
        x1 = landmarks[0]
        y1 = landmarks[1]
        x2 = landmarks[2]
        y2 = landmarks[3]
        
        x11 = landmarks2[0]
        y11 = landmarks2[1]
        x22 = landmarks2[2]
        y22 = landmarks2[3]
        box_h = y2 - y1
        box_w = x2 - x1
        bbox = patches.Rectangle((x1, y1), box_w, box_h,
             linewidth=2, facecolor='none',edgecolor = "white")
        
        if int(y1-(box_h/2))<0 and int(x1-(box_w/2))<0:
          croppedImage1 = img[0:int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[0:int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedX1 = 0
          croppedY1 = 0
        elif int(y1-(box_h/2))<0:
          croppedImage1 = img[0:int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[0:int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedY1 = 0
          croppedX1 = x1-box_w/2
        elif int(x1-(box_w/2))<0:
          croppedImage1 = img[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedX1 = 0
          croppedY1 = y1-box_h/2
        elif int(y1-(box_h/2))>=0 and int(x1-(box_w/2))>=0:
          croppedImage1 = img[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedX1 = x1-box_w/2
          croppedY1 = y1-box_h/2
        
  
        croppedX2 = x1+box_w/2
        croppedY2 = y1+box_h/2
      
        nx1 = x1 - croppedX1
        nx2 = x2 - croppedX1
        ny1 = y1 - croppedY1
        ny2 = y2 - croppedY1
      
        nx1 = nx1 * (224/len(croppedImage1[0]))
        nx2 = nx2 * (224/len(croppedImage1[0]))
        ny1 = ny1 * (224/len(croppedImage1))
        ny2 = ny2 * (224/len(croppedImage1))
      
        box_w1 = nx2-nx1
        box_h1 = ny2-ny1
      
        nx11 = x11 - croppedX1
        nx22 = x22 - croppedX1
        ny11 = y11 - croppedY1
        ny22 = y22 - croppedY1
        
        nx11 = nx11 * (224/len(croppedImage2[0]))
        nx22 = nx22 * (224/len(croppedImage2[0]))
        ny11 = ny11 * (224/len(croppedImage2))
        ny22 = ny22 * (224/len(croppedImage2))
        
        croppedX2 = croppedX2 - croppedX1
        croppedY2 = croppedY2 - croppedY1
        croppedX1 = 0
        croppedY1 = 0
  
        box_w2 = nx22-nx11
        box_h2 = ny22-ny11
      
        coordinates.append(nx1)
        coordinates.append(ny1)
        coordinates.append(nx2)
        coordinates.append(ny2)
        
        coordinates2.append(nx11)
        coordinates2.append(ny11)
        coordinates2.append(nx22)
        coordinates2.append(ny22)
        bbox2 = patches.Rectangle((nx11, ny11), box_w2, box_h2,
             linewidth=2, facecolor='none',edgecolor = "white")
        
        croppedImage1 = self.transform(croppedImage1)
        croppedImage2 = self.transform(croppedImage2)
        x = torchvision.utils.make_grid(croppedImage1, nrow=5).permute(1, 2, 0)
        y = torchvision.utils.make_grid(croppedImage2, nrow=5).permute(1, 2, 0)
#         plt.figure()
#         plt.title(video_name)
#         plt.imshow(x)
#         plt.figure()
#         plt.title(video_name)
#         plt.imshow(y)
#         plt.figure()
#         fig, ax = plt.subplots(1)
#         ax.add_patch(bbox2)
#         plt.imshow(y)
#         plt.text(nx11, y11, s=str(nx11)+","+str(ny11), 
#                 color='white', verticalalignment='top',
#                 bbox={'color': "green", 'pad': 0})
#         plt.axis('off')
#         plt.show()
        
        coordinates = torch.from_numpy(np.array(coordinates))
        coordinates2 = torch.from_numpy(np.array(coordinates2))
      
        
        sample1 = (x,y,coordinates,coordinates2)
        return sample1

In [0]:
class TestDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir 
        self.transform = transform
        

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        video_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        frame_name = os.path.join(video_name,
                                self.landmarks_frame.iloc[idx, 6])
        image = io.imread(frame_name)
        frame2_name = os.path.join(video_name,
                                self.landmarks_frame.iloc[idx, 7])
        image2 = io.imread(frame2_name)
        landmarks = self.landmarks_frame.iloc[idx, 2:6].values
        
        img = np.array(image)
        img2 = np.array(image2)
        coordinates = []
        coordinates2 = []
        x1 = landmarks[0]
        y1 = landmarks[1]
        x2 = landmarks[2]
        y2 = landmarks[3]
        
        box_h = y2 - y1
        box_w = x2 - x1
        bbox = patches.Rectangle((x1, y1), box_w, box_h,
             linewidth=2, facecolor='none',edgecolor = "white")
        
        if int(y1-(box_h/2))<0 and int(x1-(box_w/2))<0:
          croppedImage1 = img[0:int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[0:int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedX1 = 0
          croppedY1 = 0
        elif int(y1-(box_h/2))<0:
          croppedImage1 = img[0:int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[0:int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedY1 = 0
          croppedX1 = x1-box_w/2
        elif int(x1-(box_w/2))<0:
          croppedImage1 = img[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),0:int(x1+box_w+(box_w/2)),:]
          croppedX1 = 0
          croppedY1 = y1-box_h/2
        elif int(y1-(box_h/2))>=0 and int(x1-(box_w/2))>=0:
          croppedImage1 = img[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedImage2 = img2[int(y1-(box_h/2)):int(y1+box_h+(box_h/2)),int(x1-(box_w/2)):int(x1+box_w+(box_w/2)),:]
          croppedX1 = x1-box_w/2
          croppedY1 = y1-box_h/2
        
  
        croppedX2 = x1+box_w/2
        croppedY2 = y1+box_h/2
      
        nx1 = x1 - croppedX1
        nx2 = x2 - croppedX1
        ny1 = y1 - croppedY1
        ny2 = y2 - croppedY1
      
        nx1 = nx1 * (224/len(croppedImage1[0]))
        nx2 = nx2 * (224/len(croppedImage1[0]))
        ny1 = ny1 * (224/len(croppedImage1))
        ny2 = ny2 * (224/len(croppedImage1))
      
        box_w1 = nx2-nx1
        box_h1 = ny2-ny1
      
        
        
        croppedX2 = croppedX2 - croppedX1
        croppedY2 = croppedY2 - croppedY1
        croppedX1 = 0
        croppedY1 = 0
  
      
        coordinates.append(nx1)
        coordinates.append(ny1)
        coordinates.append(nx2)
        coordinates.append(ny2)
        
#         bbox2 = patches.Rectangle((nx11, ny11), box_w2, box_h2,
#              linewidth=2, facecolor='none',edgecolor = "white")
        
        croppedImage1 = self.transform(croppedImage1)
        croppedImage2 = self.transform(croppedImage2)
        x = torchvision.utils.make_grid(croppedImage1, nrow=5).permute(1, 2, 0)
        y = torchvision.utils.make_grid(croppedImage2, nrow=5).permute(1, 2, 0)
#         plt.figure()
#         plt.title(video_name)
#         plt.imshow(x)
#         plt.figure()
#         plt.title(video_name)
#         plt.imshow(y)
#         plt.figure()
#         fig, ax = plt.subplots(1)
#         ax.add_patch(bbox2)
#         plt.imshow(y)
#         plt.text(nx11, y11, s=str(nx11)+","+str(ny11), 
#                 color='white', verticalalignment='top',
#                 bbox={'color': "green", 'pad': 0})
#         plt.axis('off')
#         plt.show()
        
      
        coordinates = torch.from_numpy(np.array(coordinates))
      
        
        sample1 = (x,y,coordinates,torch.from_numpy(img2))
        return sample1

In [0]:
train_dataset = TrainDataset(csv_file='/content/drive/My Drive/mergedTrain.csv',
                                    root_dir='drive/My Drive/dataset/videos/train',transform = transforms.Compose([transforms.ToPILImage(),
                                                                                                                  transforms.Resize((224,224)),
                                                                                                                  transforms.ToTensor()]))






valid_dataset = ValidDataset(csv_file = '/content/drive/My Drive/mergedValidation.csv',root_dir='drive/My Drive/dataset/videos/val',transform = transforms.Compose([transforms.ToPILImage(),
                                                                                                                          transforms.Resize((224,224)),
                                                                                                                          transforms.ToTensor()]))




test_dataset = TestDataset(csv_file = '/content/drive/My Drive/mergedTest.csv',root_dir='drive/My Drive/dataset/videos/test',transform = transforms.Compose([transforms.ToPILImage(),
                                                                                                                             transforms.Resize((224,224)),
                                                                                                                             transforms.ToTensor()]))









validLoader = torch.utils.data.DataLoader(valid_dataset, batch_size=1,
                         shuffle=True, num_workers=4)


trainLoader = torch.utils.data.DataLoader(train_dataset, batch_size=1,
                         shuffle=True, num_workers=4)


testLoader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                         shuffle=False, num_workers=4)
 
  

In [0]:
class ConvNeuralNet(nn.Module):
    def __init__(self):
        super(ConvNeuralNet, self).__init__()
        self.fc1 = nn.Linear(1024*1*1, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 1*4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [0]:
model = models.vgg16(pretrained=True)
model = model.cuda()
model.eval()
pool_func = nn.AvgPool2d((7,7), stride=1, ceil_mode=False)

In [0]:
trainFeatures=[]

for i,data in enumerate(trainLoader):
  t0,t1,bbox_0 ,bbox2= data
  t0 = t0.cuda()
  t1 = t1.cuda()
  reshaped = t0.permute(0,3,1,2)
  reshaped2 = t1.permute(0,3,1,2)
  featureT0 = (pool_func(model.features(reshaped))).reshape(1,512)
  featureT1 = (pool_func(model.features(reshaped))).reshape(1,512)
  concatted = torch.cat((featureT0[0],featureT1[0]),0).reshape(1,1024)
  trainFeatures.append([concatted.data.cpu(),bbox_0,bbox2])
  del t0,t1,bbox_0,reshaped,reshaped2,featureT0,featureT1

In [0]:
validFeatures=[]

for k,datas in enumerate(validLoader):
  t0v,t1v,bbox_0v ,bbox2v= datas
  t0v = t0v.cuda()
  t1v = t1v.cuda()
  reshaped = t0v.permute(0,3,1,2)
  reshaped2 = t1v.permute(0,3,1,2)
  featureT0 = (pool_func(model.features(reshaped))).reshape(1,512)
  featureT1 = (pool_func(model.features(reshaped))).reshape(1,512)
  concattedv = torch.cat((featureT0[0],featureT1[0]),0).reshape(1,1024)
  validFeatures.append([concattedv.data.cpu(),bbox_0v,bbox2v])
  del t0v,t1v,bbox_0v,reshaped,reshaped2,featureT0,featureT1

In [240]:
print(len(validFeatures[0][0][0]))

1024


In [0]:
def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
    best_model_wts = copy.deepcopy(vgg.state_dict())
    best_loss = 999999999999999999
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    loss_graph_train = []
    loss_graph_valid = []
    
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        vgg.train(True)
        
        for i in trainFeatures:
                
            feature = i[0].float()
            label = i[2].float()
            
            feature = feature.cuda()
            label = label.cuda()
            optimizer.zero_grad()
            
            outputs = vgg(feature)
            
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()
            
            loss_train += loss.data
            
            del feature, label, outputs
            torch.cuda.empty_cache()
            
        
        print()
        avg_loss = loss_train  / len(trainFeatures)
        loss_graph_train.append(avg_loss.data.cpu().item())
        vgg.train(False)
        vgg.eval()
            
        for i in validFeatures:
                
            feature = i[0].float()
            label = i[2].float()
            
            feature = feature.cuda()
            label = label.cuda()
            
            optimizer.zero_grad()
            
            outputs = vgg(feature)
            loss = criterion(outputs, label)
            loss_val += loss.data
            del feature, label, outputs
            torch.cuda.empty_cache()
        
        avg_loss_val = loss_val / len(validFeatures)
        
        
        loss_graph_valid.append(avg_loss_val.data.cpu().item())
        
        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print('-' * 10)
        print()
        
        if avg_loss_val < best_loss:
            best_loss = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())
    plt.figure(figsize=(8, 6))
    epochs = []
    for i in range(num_epochs):
      epochs.append(i+1)
    plot1, = plt.plot(np.array(epochs),np.array(loss_graph_train))
    plot2, = plt.plot(np.array(epochs),np.array(loss_graph_valid))
    plt.legend([plot1, plot2], ["train_loss", "validation_loss"])
    plt.xlabel('Epoch')
    plt.ylabel('Average Negative Log Likelihood')
    plt.title('Training and Validation Losses')
        
    print()
    print("Best loss: {:.4f}".format(best_loss))
    vgg.load_state_dict(best_model_wts)
    return vgg

In [0]:
x = ConvNeuralNet()
x.cuda()
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(x.parameters(), lr=0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size = 4, gamma = 0.1)
train_model(x,criterion,optimizer,exp_lr_scheduler,10)

In [0]:
testFeatures=[]

for k,datas in enumerate(testLoader):
  t0c,t1c,bbox_0c= datas
  t0c = t0c.cuda()
  t1c = t1c.cuda()
  reshaped = t0c.permute(0,3,1,2)
  reshaped2 = t1c.permute(0,3,1,2)
  featureT0 = (pool_func(model.features(reshaped))).reshape(1,512)
  featureT1 = (pool_func(model.features(reshaped))).reshape(1,512)
  concattedc = torch.cat((featureT0[0],featureT1[0]),0).reshape(1,1024)
  testFeatures.append([concattedc.data.cpu(),bbox_0c])
  del t0c,t1c,bbox_0c,reshaped,reshaped2,featureT0,featureT1

In [0]:
def eval_model(vgg, criterion):
    outputs = []
    avg_loss = 0
    loss_test = 0
    loss_val = 0
    print("Evaluation model")
    print('-' * 10)
    vgg.train(False)
    vgg.eval()
    for i in testFeatures:
        temp= []
        feature = i[0].float()
        
        labels = i[1].float()
        
        feature = feature.cuda()
        labels = labels.cuda()
        output = vgg(feature)
        
        loss = criterion(output,labels)
            
        loss_val += loss.data
        
        outputs.append(output.data.cpu().numpy())
        
        del feature, output,labels
        torch.cuda.empty_cache()
        
    avg_loss_val = loss_val / len(testFeatures)
    
    print()
    print("Avg loss (test): {:.4f}".format(avg_loss_val))
    print('-' * 10)
    return outputs
    


In [0]:
coord = eval_model(x,criterion)

In [0]:
for i in range(70,140):
  coords = coord[i]
  im = testLoader.dataset[i][3]
  x1 = coords[0][0] * len(im[0])/224
  y1 = coords[0][1] * len(im)/224
  x2 = coords[0][2] * len(im[0])/224
  y2 = coords[0][3] * len(im)/224
  bbox2 = patches.Rectangle((x1, y1), x2-x1, y2-y1,
             linewidth=2, facecolor='none',edgecolor = "white")
  plt.figure()
  fig, ax = plt.subplots(1)
  ax.add_patch(bbox2)
  plt.imshow(im)
  plt.text(x1, y1, s=str(x1)+","+str(y1), 
                color='white', verticalalignment='top',
                bbox={'color': "green", 'pad': 0})
  plt.axis('off')
  plt.savefig("/content/drive/My Drive/image/{}.jpg".format(i))
  plt.show()
  plt.close(fig)